# CDC

## Publicaciones

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd  # Importar pandas para manejar los datos en formato DataFrame

# Crear el driver
def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

# Función para presionar el botón "Load More" repetidamente
def load_more_until_stable(driver, load_more_button_xpath, articles_xpath):
    initial_count = 0
    while True:
        # Hacer scroll hacia abajo para cargar más contenido
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Esperar que cargue el contenido al hacer scroll

        # Intentar encontrar los artículos y obtener la cantidad actual
        articles = driver.find_elements(By.XPATH, articles_xpath)
        current_count = len(articles)
        print(f"Se han encontrado {current_count} artículos.")
        
        if current_count == initial_count:
            # Si no hay nuevos artículos, terminamos
            print("No se han cargado más artículos. Terminando.")
            break
        else:
            initial_count = current_count  # Actualizar la cantidad de artículos

        try:
            # Esperar hasta que el botón "Load More" sea visible y clickeable
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, load_more_button_xpath))
            )
            load_more_button.click()  # Hacer clic en el botón
            time.sleep(3)  # Esperar un poco para que cargue el nuevo contenido
        except Exception as e:
            print(f"Error durante el clic o carga: {e}")
            break

# Función para extraer los artículos
def extract_articles(driver, articles_xpath):
    articles = driver.find_elements(By.XPATH, articles_xpath)
    print(f"Total de artículos encontrados para extraer: {len(articles)}")
    articles_data = []
    
    for article in articles:
        try:
            # Extraer título
            title = article.find_element(By.XPATH, ".//h2[@class='cs-entry__title']/a").text
            link = article.find_element(By.XPATH, ".//h2[@class='cs-entry__title']/a").get_attribute("href")
            
            # Extraer fecha
            date = article.find_element(By.XPATH, ".//div[@class='cs-entry__post-meta']//div[@class='cs-meta-date']").text
            
            # Extraer la imagen (si es necesario)
            img_url = article.find_element(By.XPATH, ".//div[@class='cs-entry__outer']//img").get_attribute("src")
            
            # Agregar los datos del artículo al listado
            articles_data.append({
                'title': title,
                'link': link,
                'date': date,
                'image_url': img_url
            })
        except Exception as e:
            print(f"Error al extraer datos de un artículo: {e}")
    
    # Convertir los datos a un DataFrame de pandas
    return pd.DataFrame(articles_data)

# Main script
def main():
    # Crear el driver (sin modo headless para ver la ejecución)
    driver = create_driver(headless=False)
    
    # URL base del sitio
    url = "https://www.cdc.cl/publicaciones/"
    
    # Acceder al sitio
    driver.get(url)
    
    # Esperar un poco para que se cargue la página
    time.sleep(3)
    
    # Localizar y cargar más artículos
    load_more_button_xpath = "//button[@class='cs-load-more']"  # Ajusta este XPath si es necesario
    articles_xpath = "//div[@class='cs-posts-area__outer']//article[contains(@class, 'cs-entry')]"

    # Cargar más artículos hasta que no haya cambios
    load_more_until_stable(driver, load_more_button_xpath, articles_xpath)
    
    # Extraer los artículos y devolver el DataFrame
    cdc = extract_articles(driver, articles_xpath)
    
    # Cerrar el navegador después de terminar
    driver.quit()

    # Retornar el DataFrame
    return cdc

# Ejecutar el script
if __name__ == "__main__":
    pub = main()  # Llamar a la función y almacenar el DataFrame resultante en "cdc"

pub

Se han encontrado 10 artículos.
Se han encontrado 20 artículos.
Se han encontrado 30 artículos.
Se han encontrado 40 artículos.
Se han encontrado 50 artículos.
Se han encontrado 60 artículos.
Se han encontrado 70 artículos.
Se han encontrado 80 artículos.
Se han encontrado 90 artículos.
Se han encontrado 90 artículos.
No se han cargado más artículos. Terminando.
Total de artículos encontrados para extraer: 90
Error al extraer datos de un artículo: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class='cs-entry__outer']//img"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B28380D5+2992373]
	(No symbol) [0x00007FF6B24CBFD0]
	(No symbol) [0x00007FF6B236590A]
	(No symbol) [0x00007FF6B23B926E]
	(No symbol) [0x00007FF6B23B955C]
	(No symbol) [0x00007FF6B23AC6CC]
	(No symbo

,title,link,date,image_url
0,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,08/04/2023,https://www.cdc.cl/wp-content/uploads/2023/08/...
1,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,04/21/2023,https://www.cdc.cl/wp-content/uploads/2023/04/...
2,Presentamos nuevo número de la revista “Democr...,https://www.cdc.cl/publicaciones/el-centro-dem...,03/29/2023,https://www.cdc.cl/wp-content/uploads/2023/03/...
3,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,01/20/2023,https://www.cdc.cl/wp-content/uploads/2023/01/...
4,El Mensaje Humanista,https://www.cdc.cl/publicaciones/el-mensaje-hu...,01/19/2023,https://www.cdc.cl/wp-content/uploads/2023/01/...
...,...,...,...,...
84,Nuevas Ideas Para Chile – Boletín Nº6,https://www.cdc.cl/publicaciones/nuevas-ideas-...,05/24/2019,https://www.cdc.cl/wp-content/uploads/2019/05/...
85,Nuevas Ideas Para Chile – Boletín Nº5,https://www.cdc.cl/publicaciones/nuevas-ideas-...,05/03/2019,https://www.cdc.cl/wp-content/uploads/2019/05/...
86,Nuevas Ideas Para Chile – Boletín Nº4,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/29/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
87,Nuevas Ideas Para Chile – Boletín Nº3,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/18/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...


## CDC

In [11]:
# Main script
def main_cdc():
    # Crear el driver (sin modo headless para ver la ejecución)
    driver = create_driver(headless=False)
    
    # URL base del sitio
    url = "https://www.cdc.cl/cdc/"
    
    # Acceder al sitio
    driver.get(url)
    
    # Esperar un poco para que se cargue la página
    time.sleep(3)
    
    # Localizar y cargar más artículos
    load_more_button_xpath = "//button[@class='cs-load-more']"  # Ajusta este XPath si es necesario
    articles_xpath = "//div[@class='cs-posts-area__outer']//article[contains(@class, 'cs-entry')]"

    # Cargar más artículos hasta que no haya cambios
    load_more_until_stable(driver, load_more_button_xpath, articles_xpath)
    
    # Extraer los artículos y devolver el DataFrame
    cdc = extract_articles(driver, articles_xpath)
    
    # Cerrar el navegador después de terminar
    driver.quit()

    # Retornar el DataFrame
    return cdc

# Ejecutar el script
if __name__ == "__main__":
    cdc = main_cdc()  # Llamar a la función y almacenar el DataFrame resultante en "cdc"
    
cdc

Se han encontrado 10 artículos.
Se han encontrado 20 artículos.
Se han encontrado 25 artículos.
Error durante el clic o carga: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B28380D5+2992373]
	(No symbol) [0x00007FF6B24CBFD0]
	(No symbol) [0x00007FF6B236590A]
	(No symbol) [0x00007FF6B23B926E]
	(No symbol) [0x00007FF6B23B955C]
	(No symbol) [0x00007FF6B24027D7]
	(No symbol) [0x00007FF6B23DF3AF]
	(No symbol) [0x00007FF6B23FF584]
	(No symbol) [0x00007FF6B23DF113]
	(No symbol) [0x00007FF6B23AA918]
	(No symbol) [0x00007FF6B23ABA81]
	GetHandleVerifier [0x00007FF6B2896A2D+3379789]
	GetHandleVerifier [0x00007FF6B28AC32D+3468109]
	GetHandleVerifier [0x00007FF6B28A0043+3418211]
	GetHandleVerifier [0x00007FF6B262C78B+847787]
	(No symbol) [0x00007FF6B24D757F]
	(No symbol) [0x00007FF6B24D2FC4]
	(No symbol) [0x00007FF6B24D315D]
	(No symbol) [0x00007FF6B24C2979]
	BaseThreadInitThunk [0x00007FF98688259D+29]
	RtlUserThreadStart [0x00007FF9885EAF38+40]

Total de artículos encontrados para extraer: 2

,title,link,date,image_url
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,07/10/2024,https://www.cdc.cl/wp-content/uploads/2024/07/...
1,CDC inicia trabajo con Comisión de Modernizaci...,https://www.cdc.cl/cdc/cdc-inicia-trabajo-con-...,07/16/2023,https://www.cdc.cl/wp-content/uploads/2023/07/...
2,Análisis del proceso constitucional: reunión c...,https://www.cdc.cl/cdc/analisis-del-proceso-co...,07/03/2023,https://www.cdc.cl/wp-content/uploads/2023/07/...
3,En CDC se realiza taller de análisis electoral...,https://www.cdc.cl/cdc/tomo-lugar-en-cdc-talle...,06/22/2023,https://www.cdc.cl/wp-content/uploads/2023/06/...
4,Jornada de reflexión con el equipo de CDC,https://www.cdc.cl/cdc/jornada-de-reflexion-co...,06/15/2023,https://www.cdc.cl/wp-content/uploads/2023/06/...
5,CDC participa de importante seminario de ASEMU...,https://www.cdc.cl/cdc/cdc-participa-de-import...,05/25/2023,https://www.cdc.cl/wp-content/uploads/2023/05/...
6,CDC formaliza trabajo con el National Democrat...,https://www.cdc.cl/cdc/cdc-formaliza-trabajo-c...,04/28/2023,https://www.cdc.cl/wp-content/uploads/2023/04/...
7,[Galería] Encuentro Estudiantil de Verano 2023,https://www.cdc.cl/cdc/encuentro-estudiantil-d...,02/06/2023,https://www.cdc.cl/wp-content/uploads/2023/02/...
8,Ideas y Propuestas de la Democracia Cristiana ...,https://www.cdc.cl/cdc/ideas-y-propuestas-de-l...,12/14/2022,https://www.cdc.cl/wp-content/uploads/2022/12/...
9,Conversación con Mario Desbordes,https://www.cdc.cl/cdc/conversacion-con-mario-...,11/04/2022,https://www.cdc.cl/wp-content/uploads/2022/11/...


## Otras

In [13]:
# Main script
def main_otras():
    # Crear el driver (sin modo headless para ver la ejecución)
    driver = create_driver(headless=False)
    
    # URL base del sitio
    url = "https://www.cdc.cl/otras/"
    
    # Acceder al sitio
    driver.get(url)
    
    # Esperar un poco para que se cargue la página
    time.sleep(3)
    
    # Localizar y cargar más artículos
    load_more_button_xpath = "//button[@class='cs-load-more']"  # Ajusta este XPath si es necesario
    articles_xpath = "//div[@class='cs-posts-area__outer']//article[contains(@class, 'cs-entry')]"

    # Cargar más artículos hasta que no haya cambios
    load_more_until_stable(driver, load_more_button_xpath, articles_xpath)
    
    # Extraer los artículos y devolver el DataFrame
    cdc = extract_articles(driver, articles_xpath)
    
    # Cerrar el navegador después de terminar
    driver.quit()

    # Retornar el DataFrame
    return cdc

# Ejecutar el script
if __name__ == "__main__":
    otras = main_otras()  # Llamar a la función y almacenar el DataFrame resultante en "cdc"
    
otras

Se han encontrado 2 artículos.
Error durante el clic o carga: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B28380D5+2992373]
	(No symbol) [0x00007FF6B24CBFD0]
	(No symbol) [0x00007FF6B236590A]
	(No symbol) [0x00007FF6B23B926E]
	(No symbol) [0x00007FF6B23B955C]
	(No symbol) [0x00007FF6B24027D7]
	(No symbol) [0x00007FF6B23DF3AF]
	(No symbol) [0x00007FF6B23FF584]
	(No symbol) [0x00007FF6B23DF113]
	(No symbol) [0x00007FF6B23AA918]
	(No symbol) [0x00007FF6B23ABA81]
	GetHandleVerifier [0x00007FF6B2896A2D+3379789]
	GetHandleVerifier [0x00007FF6B28AC32D+3468109]
	GetHandleVerifier [0x00007FF6B28A0043+3418211]
	GetHandleVerifier [0x00007FF6B262C78B+847787]
	(No symbol) [0x00007FF6B24D757F]
	(No symbol) [0x00007FF6B24D2FC4]
	(No symbol) [0x00007FF6B24D315D]
	(No symbol) [0x00007FF6B24C2979]
	BaseThreadInitThunk [0x00007FF98688259D+29]
	RtlUserThreadStart [0x00007FF9885EAF38+40]

Total de artículos encontrados para extraer: 2


,title,link,date,image_url
0,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,04/17/2024,https://www.cdc.cl/wp-content/uploads/2024/04/...
1,¿Por qué el Comunitarismo? Porque es una idea ...,https://www.cdc.cl/otras/por-que-el-comunitari...,06/03/2022,https://www.cdc.cl/wp-content/uploads/2021/04/...


## Centro de documentación

Aquí hay más artículos, algunos están repetidos pero hay más en total.

In [14]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)
        
        # Realizar scroll hasta el final de la página (si es necesario)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Esperamos un poco para que se cargue el contenido

        # Obtener todos los artículos dentro de los divs con la clase 'pt-cv-content-item'
        articles = driver.find_elements(By.CSS_SELECTOR, "div.pt-cv-content-item")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, ".pt-cv-thumb-wrapper img")
                image_url = img.get_attribute("src")
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, ".pt-cv-title a")
                title = title_element.text.strip()
                link = title_element.get_attribute("href")
            except:
                title = "No disponible"
                link = "No disponible"

            # Categorías (pueden haber varias, las extraemos todas)
            categories = []
            try:
                category_elements = article.find_elements(By.CSS_SELECTOR, ".pt-cv-taxoterm a")
                for category in category_elements:
                    categories.append(category.text.strip())
            except:
                categories = ["No especificada"]

            # Contenido (extracto del texto)
            try:
                content = article.find_element(By.CSS_SELECTOR, ".pt-cv-content").text.strip()
            except:
                content = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Categorías': ", ".join(categories),  # Unimos las categorías con coma
                'URL Imagen': image_url,
                'Contenido': content,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()
    
def extract_articles_from_multiple_pages(driver, base_url, max_pages=10, existing_df=None):
    """
    Extrae información de artículos desde varias páginas de un sitio web y concatena los resultados en un DataFrame.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        base_url (str): URL base de la página de artículos (sin el parámetro de página).
        max_pages (int, optional): Número máximo de páginas a recorrer (por defecto 10). 
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas.
    """
    all_data = existing_df if existing_df is not None else pd.DataFrame()

    for page_num in range(1, max_pages + 1):
        try:
            # Construir la URL con el número de página
            url = f"{base_url}?_page={page_num}"
            print(f"Extrayendo artículos de {url}...")

            # Extraer los artículos de la página actual
            page_df = extract_articles_from_url(driver, url, existing_df=None)

            # Concatenar los artículos extraídos a los datos existentes
            all_data = pd.concat([all_data, page_df], ignore_index=True)
            
            # Esperar un poco entre las páginas para no sobrecargar el servidor
            time.sleep(3)

            # Verificar si la página está vacía o no contiene artículos (en ese caso, detener la iteración)
            if page_df.empty:
                print("No se encontraron más artículos en esta página. Terminando la extracción.")
                break

        except Exception as e:
            print(f"Error en la página {page_num}: {e}")
            continue  # Si ocurre un error en una página, seguir con la siguiente

    return all_data

In [15]:
# Crear un driver
driver = create_driver(headless=False)

# Definir la URL base sin el parámetro de página
base_url = "https://www.cdc.cl/centro-de-documentacion/"

# Extraer los artículos de las primeras 5 páginas
cdoc = extract_articles_from_multiple_pages(driver, base_url, max_pages=7)

# Ver los primeros registros extraídos
cdoc

Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=1...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=2...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=3...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=4...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=5...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=6...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=7...


,Título,Enlace,Categorías,URL Imagen,Contenido,Fecha Consulta
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,"Humanismo Cristiano, Otros, Reflexión y debate...",https://www.cdc.cl/wp-content/uploads/2024/07/...,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,2025-01-09 14:32:23
1,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,"Reflexión y debate, seguridad y derechos humanos",https://www.cdc.cl/wp-content/uploads/2024/04/...,"Desde el 2004, nuestro país cuenta con un Sist...",2025-01-09 14:32:23
2,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,Humanismo Cristiano,https://www.cdc.cl/wp-content/uploads/2023/08/...,"Maritain fue un insigne filósofo, pensador, es...",2025-01-09 14:32:23
3,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,Humanismo Cristiano,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Hace más de una década, cuando se inauguró el ...",2025-01-09 14:32:23
4,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-09 14:32:23
...,...,...,...,...,...,...
112,Un Chile abierto: propuesta para una nueva ley...,https://www.cdc.cl/publicaciones/un-chile-abie...,Perfeccionamiento de la Democracia,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Autor: Centro Democracia y Comunidad, de la Or...",2025-01-09 14:33:41
113,El desafío de la ciudad y calidad de vida,https://www.cdc.cl/publicaciones/el-desafio-de...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Iván Poduje y Marcelo Ruiz Año: 2013 Br...,2025-01-09 14:33:42
114,La Solidaridad social como carencia y oportuni...,https://www.cdc.cl/publicaciones/la-solidarida...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Felipe Zaldivia Fuenzalida Año: 2014 Br...,2025-01-09 14:33:42
115,Hombre-Medio Ambiente: Fundamentos para una re...,https://www.cdc.cl/publicaciones/hombre-medio-...,"Humanismo Cristiano, Medio Ambiente",https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Yasmina Viera Bernal Año: 2014 Breve de...,2025-01-09 14:33:42


# Concatenación

In [19]:
lista_df = [cdc, pub, otras] 

In [20]:
import pandas as pd

def concatenar_dataframes_con_reemplazo(lista_df):
    # Reemplazo de nombres de columnas
    for i, df in enumerate(lista_df):
        # Reemplazo de las columnas 'tittle' por 'Titulo' y 'link' por 'Enlace'
        df.columns = df.columns.str.replace('title', 'Título', case=False)
        df.columns = df.columns.str.replace('link', 'Enlace', case=False)
        df.columns = df.columns.str.replace('date', 'Fecha', case=False)
    
    # Concatenar todos los dataframes
    df_concatenado = pd.concat(lista_df, ignore_index=True, sort=False)
    
    return df_concatenado

df_resultado = concatenar_dataframes_con_reemplazo(lista_df)

# Mostrar el dataframe resultante
df_resultado

,Título,Enlace,Fecha,image_url
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,07/10/2024,https://www.cdc.cl/wp-content/uploads/2024/07/...
1,CDC inicia trabajo con Comisión de Modernizaci...,https://www.cdc.cl/cdc/cdc-inicia-trabajo-con-...,07/16/2023,https://www.cdc.cl/wp-content/uploads/2023/07/...
2,Análisis del proceso constitucional: reunión c...,https://www.cdc.cl/cdc/analisis-del-proceso-co...,07/03/2023,https://www.cdc.cl/wp-content/uploads/2023/07/...
3,En CDC se realiza taller de análisis electoral...,https://www.cdc.cl/cdc/tomo-lugar-en-cdc-talle...,06/22/2023,https://www.cdc.cl/wp-content/uploads/2023/06/...
4,Jornada de reflexión con el equipo de CDC,https://www.cdc.cl/cdc/jornada-de-reflexion-co...,06/15/2023,https://www.cdc.cl/wp-content/uploads/2023/06/...
...,...,...,...,...
111,Nuevas Ideas Para Chile – Boletín Nº4,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/29/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
112,Nuevas Ideas Para Chile – Boletín Nº3,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/18/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
113,Nuevas Ideas para Chile – Boletín Nº2,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/12/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
114,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,04/17/2024,https://www.cdc.cl/wp-content/uploads/2024/04/...


Extraer info de cdoc, principalmente la fecha

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

def extract_ficha_info(driver, url):
    """
    Extrae la información de la ficha de un documento específico
    """
    try:
        print(f"Procesando: {url}")
        driver.get(url)
        
        # Dar tiempo para que cargue el contenido
        time.sleep(1)
        
        # Inicializar diccionario para almacenar la información
        info = {}
        
        # Buscar el contenido completo de la ficha
        try:
            contenido_ficha = driver.find_element(By.CLASS_NAME, "kt-inside-inner-col")
            texto_ficha = contenido_ficha.text
        except NoSuchElementException:
            texto_ficha = ""
        
        # Mapeo de campos a buscar
        campos = {
            "Título": "",
            "Autor(es)": "taxonomy-autor",
            "Colección": "taxonomy-coleccion",
            "Tema(s)": "taxonomy-tema",
            "Fecha": "",
            "Palabras clave": "taxonomy-post_tag"
        }
        
        # Extraer información para cada campo
        for campo, clase in campos.items():
            try:
                if campo == "Título":
                    try:
                        # Usar CSS Selector para buscar el título
                        titulo_elemento = driver.find_element(By.CSS_SELECTOR, "div.kadence-column9093_f97a67-dd .kt-inside-inner-col p")
                        info["Título"] = titulo_elemento.text.strip()
                    except NoSuchElementException:
                        info["Título"] = "No encontrado"
                elif clase:  # Si tiene una clase específica
                    elemento = driver.find_element(By.CLASS_NAME, clase)
                    info[campo] = elemento.text.strip()
                else:  # Para campos sin clase específica
                    # Buscar el campo por su etiqueta dentro del texto de la ficha
                    if campo == "Fecha":
                        # Intentar obtener la fecha desde el HTML tradicional
                        if campo in texto_ficha:
                            start_index = texto_ficha.find(campo + ":") + len(campo) + 1
                            if start_index != -1:
                                end_index = texto_ficha.find("\n", start_index)
                                if end_index == -1:  # Si no hay salto de línea, tomamos hasta el final
                                    valor = texto_ficha[start_index:].strip()
                                else:
                                    valor = texto_ficha[start_index:end_index].strip()
                                info[campo] = valor
                            else:
                                info[campo] = "No encontrado"
                        else:
                            info[campo] = "No encontrado"
                    
                    # Si la fecha no se encuentra en el texto, intentar buscarla en el elemento con la clase 'cs-meta-date'
                    if info.get("Fecha") == "No encontrado":
                        try:
                            fecha_elemento = driver.find_element(By.CSS_SELECTOR, "div.cs-entry__header-info div.cs-entry__post-meta div.cs-meta-date")
                            info["Fecha"] = fecha_elemento.text.strip()
                        except NoSuchElementException:
                            info["Fecha"] = "No encontrado"
                
            except NoSuchElementException:
                info[campo] = "No encontrado"
                
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        return {campo: "Error" for campo in campos.keys()}
    
    info = {key.replace('Título', 'Título ficha'): value for key, value in info.items()}
    
    return info

def process_dataframe(df, headless=True):
    """
    Procesa el dataframe completo, extrayendo información para cada enlace
    """
    # Crear una copia del dataframe original
    df_nuevo = df.copy()
    
    # Inicializar el navegador con opción headless
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')  # Modo sin interfaz gráfica
    driver = webdriver.Chrome(options=options)
    
    try:
        # Procesar cada enlace
        resultados = []
        for url in df['Enlace']:
            info = extract_ficha_info(driver, url)
            resultados.append(info)
        
        # Convertir resultados a dataframe y unir con el original
        df_info = pd.DataFrame(resultados)
        df_nuevo = pd.concat([df_nuevo, df_info], axis=1)
        
    finally:
        driver.quit()
    
    return df_nuevo

# Ejemplo de uso:
cdoc2 = process_dataframe(cdoc, headless=False)
cdoc2

Procesando: https://www.cdc.cl/cdc/es-necesario-un-enfoque-integral-de-la-gestion-de-movilidad-humana-a-nivel-de-estado-con-los-tres-niveles-de-gobierno-nacional-regional-y-local/
Procesando: https://www.cdc.cl/otras/crisis-de-seguridad-la-inteligencia-como-herramienta-indispensable/
Procesando: https://www.cdc.cl/publicaciones/historia/en-memoria-de-jacques-maritain/
Procesando: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-23/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-22/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-21/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-20/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-19/
Procesando: https://www.cdc.cl/publicaciones/boletin-legislativo-18/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-17/
Procesando: http

,Título,Enlace,Categorías,URL Imagen,Contenido,Fecha Consulta,Título ficha,Autor(es),Colección,Tema(s),Fecha,Palabras clave
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,"Humanismo Cristiano, Otros, Reflexión y debate...",https://www.cdc.cl/wp-content/uploads/2024/07/...,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,2025-01-09 14:32:23,No encontrado,No encontrado,No encontrado,No encontrado,07/10/2024,No encontrado
1,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,"Reflexión y debate, seguridad y derechos humanos",https://www.cdc.cl/wp-content/uploads/2024/04/...,"Desde el 2004, nuestro país cuenta con un Sist...",2025-01-09 14:32:23,No encontrado,No encontrado,No encontrado,No encontrado,04/17/2024,No encontrado
2,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,Humanismo Cristiano,https://www.cdc.cl/wp-content/uploads/2023/08/...,"Maritain fue un insigne filósofo, pensador, es...",2025-01-09 14:32:23,,No encontrado,No encontrado,Humanismo Cristiano,,"Análisis político, Ideas en comunidad, Portada"
3,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,Humanismo Cristiano,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Hace más de una década, cuando se inauguró el ...",2025-01-09 14:32:23,,No encontrado,No encontrado,Humanismo Cristiano,,Portada
4,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-09 14:32:23,Boletín legislativo N° 24,Centro Democracia y Comunidad,La Semana en la Cámara,Legislativo y Constitucional,,"Boletín Legislativo, derechos de aseo, ENAP, T..."
...,...,...,...,...,...,...,...,...,...,...,...,...
112,Un Chile abierto: propuesta para una nueva ley...,https://www.cdc.cl/publicaciones/un-chile-abie...,Perfeccionamiento de la Democracia,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Autor: Centro Democracia y Comunidad, de la Or...",2025-01-09 14:33:41,,No encontrado,No encontrado,Perfeccionamiento de la Democracia,,No encontrado
113,El desafío de la ciudad y calidad de vida,https://www.cdc.cl/publicaciones/el-desafio-de...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Iván Poduje y Marcelo Ruiz Año: 2013 Br...,2025-01-09 14:33:42,,No encontrado,No encontrado,Economía con equidad,,No encontrado
114,La Solidaridad social como carencia y oportuni...,https://www.cdc.cl/publicaciones/la-solidarida...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Felipe Zaldivia Fuenzalida Año: 2014 Br...,2025-01-09 14:33:42,,No encontrado,No encontrado,Economía con equidad,,No encontrado
115,Hombre-Medio Ambiente: Fundamentos para una re...,https://www.cdc.cl/publicaciones/hombre-medio-...,"Humanismo Cristiano, Medio Ambiente",https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Yasmina Viera Bernal Año: 2014 Breve de...,2025-01-09 14:33:42,,No encontrado,No encontrado,"Humanismo Cristiano, Medio Ambiente",,No encontrado


In [28]:
df_resultado

,Título,Enlace,Fecha,image_url
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,07/10/2024,https://www.cdc.cl/wp-content/uploads/2024/07/...
1,CDC inicia trabajo con Comisión de Modernizaci...,https://www.cdc.cl/cdc/cdc-inicia-trabajo-con-...,07/16/2023,https://www.cdc.cl/wp-content/uploads/2023/07/...
2,Análisis del proceso constitucional: reunión c...,https://www.cdc.cl/cdc/analisis-del-proceso-co...,07/03/2023,https://www.cdc.cl/wp-content/uploads/2023/07/...
3,En CDC se realiza taller de análisis electoral...,https://www.cdc.cl/cdc/tomo-lugar-en-cdc-talle...,06/22/2023,https://www.cdc.cl/wp-content/uploads/2023/06/...
4,Jornada de reflexión con el equipo de CDC,https://www.cdc.cl/cdc/jornada-de-reflexion-co...,06/15/2023,https://www.cdc.cl/wp-content/uploads/2023/06/...
...,...,...,...,...
111,Nuevas Ideas Para Chile – Boletín Nº4,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/29/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
112,Nuevas Ideas Para Chile – Boletín Nº3,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/18/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
113,Nuevas Ideas para Chile – Boletín Nº2,https://www.cdc.cl/publicaciones/nuevas-ideas-...,04/12/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
114,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,04/17/2024,https://www.cdc.cl/wp-content/uploads/2024/04/...


In [27]:
lista_df2 = [cdoc, df_resultado]
df_cdc = pd.concat(lista_df2, ignore_index=True, sort=False)
df_cdc

,Título,Enlace,Categorías,URL Imagen,Contenido,Fecha Consulta,Fecha,image_url
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,"Humanismo Cristiano, Otros, Reflexión y debate...",https://www.cdc.cl/wp-content/uploads/2024/07/...,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,2025-01-09 14:32:23,NaN,NaN
1,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,"Reflexión y debate, seguridad y derechos humanos",https://www.cdc.cl/wp-content/uploads/2024/04/...,"Desde el 2004, nuestro país cuenta con un Sist...",2025-01-09 14:32:23,NaN,NaN
2,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,Humanismo Cristiano,https://www.cdc.cl/wp-content/uploads/2023/08/...,"Maritain fue un insigne filósofo, pensador, es...",2025-01-09 14:32:23,NaN,NaN
3,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,Humanismo Cristiano,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Hace más de una década, cuando se inauguró el ...",2025-01-09 14:32:23,NaN,NaN
4,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-09 14:32:23,NaN,NaN
...,...,...,...,...,...,...,...,...
228,Nuevas Ideas Para Chile – Boletín Nº4,https://www.cdc.cl/publicaciones/nuevas-ideas-...,NaN,NaN,NaN,NaN,04/29/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
229,Nuevas Ideas Para Chile – Boletín Nº3,https://www.cdc.cl/publicaciones/nuevas-ideas-...,NaN,NaN,NaN,NaN,04/18/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
230,Nuevas Ideas para Chile – Boletín Nº2,https://www.cdc.cl/publicaciones/nuevas-ideas-...,NaN,NaN,NaN,NaN,04/12/2019,https://www.cdc.cl/wp-content/uploads/2019/04/...
231,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,NaN,NaN,NaN,NaN,04/17/2024,https://www.cdc.cl/wp-content/uploads/2024/04/...


In [29]:
df_cdc.to_excel("cdc.xlsx", index=False)